Q6 - Sravani rachamalla - 20A1HP010

# Recommender Systems with Python  
Import Libraries

In [ ]:
import numpy as np
import pandas as pd
## loading files as Data frames, data manipulation and analysis.

## Get the Data

In [ ]:

df = pd.read_csv('https://raw.githubusercontent.com/rssmahesh1/MLAP2021Exam/main/MALP2021%20Question%20paper/ratings.csv')
# Importing the dataset

In [ ]:
df.head()
# To get the first n rows

Now let's get the movie titles

In [ ]:
movie_titles = pd.read_csv("https://raw.githubusercontent.com/rssmahesh1/MLAP2021Exam/main/MALP2021%20Question%20paper/movies.csv")
movie_titles.head()

We can merge them together

In [ ]:
df = pd.merge(df,movie_titles,on='movieId')
df.head()

EDA

Let's explore the data a bit and get a look at some of the best rated movies.

# Visualization Imports

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
%matplotlib inline

Let's create a ratings dataframe with average rating and number of ratings:

In [ ]:
df.groupby('title')['rating'].mean().sort_values(ascending=False).head()

In [ ]:
df.groupby('title')['rating'].count().sort_values(ascending=False).head()

In [ ]:
ratings = pd.DataFrame(df.groupby('title')['rating'].mean())
ratings.head()

Now set the number of ratings column:

In [ ]:
ratings['num of ratings'] = pd.DataFrame(df.groupby('title')['rating'].count())
ratings.head()

Now a few histograms:

In [ ]:
plt.figure(figsize=(10,4))
ratings['num of ratings'].hist(bins=70)

In [ ]:
plt.figure(figsize=(10,4))
ratings['rating'].hist(bins=70)

In [ ]:
sns.jointplot(x='rating',y='num of ratings',data=ratings,alpha=0.5)

Okay! Now that we have a general idea of what the data looks like, let's move on to creating a simple recommendation system

# Recommending Similar Movies

Now let's create a matrix that has the user ids on one access and the movie title on another axis. Each cell will then consist of the rating the user gave to that movie. Note there will be a lot of NaN values, because most people have not seen most of the movies

In [ ]:
moviemat = df.pivot_table(index='userId',columns='title',values='rating')
moviemat.head()

Most rated movie:


In [ ]:
ratings.sort_values('num of ratings',ascending=False).head(10)

Let's choose two movies: Forrest Gump, comedy drama and romance genre and Pulp Fiction a crime, thriller comedy and draqma movie

In [ ]:
ratings.head()

Now let's grab the user ratings for those two movies

In [ ]:
ForrestGump_user_ratings = moviemat['Forrest Gump (1994)']
PulpFiction_user_ratings = moviemat['Pulp Fiction (1994)']
ForrestGump_user_ratings.head()

We can then use corrwith() method to get correlations between two pandas series:

In [ ]:
similar_to_ForrestGump = moviemat.corrwith(ForrestGump_user_ratings)
similar_to_PulpFiction = moviemat.corrwith(PulpFiction_user_ratings)

Let's clean this by removing NaN values and using a DataFrame instead of a series

In [ ]:
corr_ForrestGump = pd.DataFrame(similar_to_ForrestGump,columns=['Correlation'])
corr_ForrestGump.dropna(inplace=True)
corr_ForrestGump.head()

Now if we sort the dataframe by correlation, we should get the most similar movies, however note that we get some results that don't really make sense. This is because there are a lot of movies only watched once by users who also watched star wars (it was the most popular movie)

In [ ]:
corr_ForrestGump.sort_values('Correlation',ascending=False).head(10)

Let's fix this by filtering out movies that have less than 100 reviews (this value was chosen based off the histogram from earlier).

In [ ]:
corr_ForrestGump = corr_ForrestGump.join(ratings['num of ratings'])
corr_ForrestGump.head()

Now sort the values and notice how the titles make a lot more sense:

In [ ]:
corr_ForrestGump[corr_ForrestGump['num of ratings']>100].sort_values('Correlation',ascending=False).head()

Now the same for the crime and thriller Pulp Fiction

In [ ]:
corr_PulpFiction = pd.DataFrame(similar_to_PulpFiction,columns=['Correlation'])
corr_PulpFiction.dropna(inplace=True)
corr_PulpFiction = corr_PulpFiction.join(ratings['num of ratings'])
corr_PulpFiction[corr_PulpFiction['num of ratings']>100].sort_values('Correlation',ascending=False).head()